In [ ]:
#!pip install -q --upgrade keras-nlp
#!pip install -q --upgrade keras

In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras_nlp
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import time

keras.mixed_precision.set_global_policy("mixed_float16")

In [2]:
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split



In [4]:


MAX_SEQUENCE_LENGTH = 128
MAX_GENERATION_LENGTH = 200

RoBERTa_PRESET = "roberta_base_en"

RANK = 4
ALPHA = 32.0

In [5]:
def get_optimizer_and_loss():
    optimizer = keras.optimizers.AdamW(
        learning_rate=5e-5,
        weight_decay=0.01,
        epsilon=1e-6,
        global_clipnorm=1.0,  # Gradient clipping.
    )
    # Exclude layernorm and bias terms from weight decay.
    optimizer.exclude_from_weight_decay(var_names=["bias"])
    optimizer.exclude_from_weight_decay(var_names=["gamma"])
    optimizer.exclude_from_weight_decay(var_names=["beta"])

    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    return optimizer, loss

In [6]:
def get_model():
  preprocessor = keras_nlp.models.RobertaPreprocessor.from_preset(
      "roberta_base_en",sequence_length=MAX_SEQUENCE_LENGTH
  )

  classifier = keras_nlp.models.RobertaClassifier.from_preset(
      "roberta_base_en",
      num_classes=3,
      preprocessor=preprocessor
  )
  return classifier

In [7]:
mnli = tfds.load("Multi_NLI")

In [8]:
train_d=mnli["train"]
valid_m=mnli["validation_matched"]
valid_nm=mnli["validation_mismatched"]

In [9]:
def data_prep_mnli(train,samp):
  x=[]
  y=[]
  check=0
  tot=len(train)
  for i in train:
    x.append((i["hypothesis"].numpy(),i["premise"].numpy()))
    y.append(i["label"].numpy())
    check+=1
    if check==samp:
      break
  return x,y

In [10]:

def data_division(x,y):
  x_12, x_3, y_12, y_3 = train_test_split(x, y, test_size=0.3333, random_state=1)

  x_1, x_2, y_1, y_2 = train_test_split(x_12, y_12, test_size=0.5, random_state=1)
  return ((x_1,y_1),(x_2,y_2),(x_3,y_3))


In [11]:
import math


class LoraLayer(keras.layers.Layer):
    def __init__(
        self,
        original_layer,
        rank=8,
        alpha=32,
        trainable=False,
        **kwargs,
    ):
        # We want to keep the name of this layer the same as the original
        # dense layer.
        original_layer_config = original_layer.get_config()
        name = original_layer_config["name"]

        kwargs.pop("name", None)

        super().__init__(name=name, trainable=trainable, **kwargs)

        self.rank = rank
        self.alpha = alpha

        self._scale = alpha / rank

        self._num_heads = original_layer_config["output_shape"][-2]
        self._hidden_dim = self._num_heads * original_layer_config["output_shape"][-1]

        # Layers.

        # Original dense layer.
        self.original_layer = original_layer
        # No matter whether we are training the model or are in inference mode,
        # this layer should be frozen.
        self.original_layer.trainable = False

        # LoRA dense layers.
        self.A = keras.layers.Dense(
            units=rank,
            use_bias=False,

            # Note: the original paper mentions that normal distribution was
            # used for initialization. However, the official LoRA implementation
            # uses "Kaiming/He Initialization".
            kernel_initializer=keras.initializers.VarianceScaling(
                scale=math.sqrt(5), mode="fan_in", distribution="uniform"
            ),
            trainable=trainable,
            name=f"lora_A",
        )
        # B has the same `equation` and `output_shape` as the original layer.
        # `equation = abc,cde->abde`, where `a`: batch size, `b`: sequence
        # length, `c`: `hidden_dim`, `d`: `num_heads`,
        # `e`: `hidden_dim//num_heads`. The only difference is that in layer `B`,
        # `c` represents `rank`.
        self.B = keras.layers.EinsumDense(
            equation=original_layer_config["equation"],
            output_shape=original_layer_config["output_shape"],
            kernel_initializer="zeros",
            trainable=trainable,
            name=f"lora_B",
        )

    def call(self, inputs):
        original_output = self.original_layer(inputs)
        lora_output = self.B(self.A(inputs)) * self._scale
        return original_output + lora_output


In [12]:
def lora(classifier):
  for layer_idx in range(classifier.backbone.num_layers):
    # Change query dense layer.
      decoder_layer = classifier.backbone.get_layer(f"transformer_layer_{layer_idx}")
      self_attention_layer = decoder_layer._self_attention_layer
    # Allow mutation to Keras layer state.
      self_attention_layer._tracker.locked = False

    # Change query dense layer.
      self_attention_layer._query_dense = LoraLayer(
        self_attention_layer._query_dense,
        rank=RANK,
        alpha=ALPHA,
        trainable=True,
    )

    # Change value dense layer.
      self_attention_layer._value_dense = LoraLayer(
        self_attention_layer._value_dense,
        rank=RANK,
        alpha=ALPHA,
        trainable=True,
    )
  for layer in classifier._flatten_layers():
    lst_of_sublayers = list(layer._flatten_layers())

    if len(lst_of_sublayers) == 1:  # "leaves of the model"
        if layer.name in ["lora_A", "lora_B"]:
          layer.trainable = True
        else:
          layer.trainable = False

  return classifier


In [13]:
def batchify(data_div,batch,n):
  li=[]
  for d in range(n):
    x=[data_div[d][0][i] for i in range(len(data_div[d][0]))]
    y=[data_div[d][1][i] for i in range(len(data_div[d][1]))]
    #print(y)
    #print(len(x))
    temp=[]
    for j in range(0,len(data_div[d][0]),batch):
      temp.append([x[j:j+batch],y[j:j+batch]])
    #print(len(temp))
    li.append(temp)
  #print(len(li))
  return zip(*li)





In [14]:
def models(n):
  liModels=[]
  for i in range(n):
    keras.backend.clear_session()
    liModels.append(get_model())
  loraModels=[]
  for i in liModels:
    keras.backend.clear_session()
    loraModels.append(lora(i))
  return loraModels

In [15]:
def A_matrix(param):
  li=[]
  for i in range(len(param)):
    if param[i][1].path.split("/")[-2] == "lora_A":
      li.append(param[i])
  return li


In [16]:
def B_matrix(param):
  li=[]
  for i in range(len(param)):
    if param[i][1].path.split("/")[-2] == "lora_B":
      li.append(param[i])
  return li

In [17]:
def get_grads(model,x,y,loss,loraModels):
  with tf.GradientTape() as tape:
    x=np.array(x)
    pred=model(loraModels[0].preprocessor(x),training=True)
    loss_value=loss(np.array(y),pred)
  grad=tape.gradient(loss_value,model.trainable_variables)
  #print(grad)
  return (loss_value,zip(grad,model.trainable_variables))

In [18]:
def projected_GD(n,lr,np_lis,epochs=20):
  li=[0]*n
  for i in range(n):
    po=np.random.normal((1))
    li[i]=tf.Variable(po,tf.float32)
  sum=0
  for i in range(epochs):
    with tf.GradientTape() as tape:
      for i in range(n):
        sum=sum+(li[i]*np_lis[i][0])
      sqn=tf.math.square(tf.norm(sum))
    grads=tape.gradient(sqn,[i for i in li])
    #print(li)
    for i in range(n):
      li[i].assign_sub(lr*grads[i])
    #print(li)
    z=[i.numpy() for i in li]
    c=projection_simplex_sort(np.array(z))
    for i in range(n):
      li[i]=tf.Variable(c[i],tf.float32)
  fin=0
  for i in range(n):
    fin=fin+(li[i]*np_lis[i][0])
  keras.backend.clear_session()
  return fin






In [19]:
def projection_simplex_sort(v, z=1):
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w

In [43]:
def train(loraModels,D,batch_size,loss,epochs,p_epochs=20,lr=0.001):
  opt={i:get_optimizer_and_loss()[0] for i in range(len(loraModels))}
  model_train_loss={}

  for l_m in range(len(loraModels)):
    tf.keras.backend.clear_session()
    model_train_loss[l_m]=tf.keras.metrics.Mean(name='mean', dtype=None)
  for epy in range(1,epochs+1):
    count=0
    tf.keras.backend.clear_session()
    print("Epoch ",epy, " out of ",epochs)
    Dsets=batchify(D,batch_size,len(loraModels))
    cut=0
    while True:
      current_batch=[]
      a=next(Dsets,None)
      count+=1
      if a is None:
        cut=1
        break
      for b in range(len(loraModels)):
        current_batch.append(a[b])
      g={}
      A_lora={}
      for d in range(len(current_batch)):
        tf.keras.backend.clear_session()
        loss_val,g[d]=get_grads(loraModels[d],current_batch[d][0],current_batch[d][1],loss,loraModels)
        li_g=list(g[d])
        model_train_loss[d].update_state(loss_val)
        A_lora[d]=A_matrix(li_g)
        #print("Hel_2")
        opt[d].apply_gradients(B_matrix(li_g))
        print("Model ",d," batch ",count)
      for i in range(len(A_lora[0])):
        arr=[]
        #up_a=[]
        for j in range(len(loraModels)):
          arr.append(A_lora[j][i])
          #up_a.append((j,A_lora[j][i]))
        p=projected_GD(len(loraModels),lr,arr,p_epochs)
        #p
        for iter in arr:
          #print(iter[1][1])
          iter[1].assign_sub(0.001*p)
        #print("Done")
    for avg_loss in range(len(loraModels)):
      print("Avg_training_loss for model ", avg_loss," in epoch ",epy, " is ", model_train_loss[avg_loss].result().numpy())
    for avg_loss in range(len(loraModels)):
      model_train_loss[avg_loss].reset_state()
  return loraModels


In [21]:
x,y=data_prep_mnli(train_d,160)
data_div=data_division(x,y)

In [22]:
loraModels=models(3)

In [24]:
_,loss=get_optimizer_and_loss()

In [ ]:
m=train(loraModels,data_div,16,loss,2)